In [65]:
from torchvision import models
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image

In [66]:
# Load model dataset
vgg16 = torch.load("vgg16.pt")
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [67]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

In [68]:
# creates a flatten layer to feed the output of the avg pool mod to the Linear mod
submodel_flatten = nn.Sequential(Flatten())

In [69]:
submodel_vgg_1 = list(vgg16.children())[:1]
submodel_vgg_1 = nn.Sequential(*submodel_vgg_1)

In [70]:
for n in submodel_vgg_1.children():
  submodel_1 = nn.Sequential(n[:6])
  submodel_2 = nn.Sequential(n[6:9])
  submodel_3 = nn.Sequential(n[9:12])
  submodel_4 = nn.Sequential(n[12:15])
  submodel_5 = nn.Sequential(n[15:18])
  submodel_6 = nn.Sequential(n[18:21])
  submodel_7 = nn.Sequential(n[21:24])
  submodel_8 = nn.Sequential(n[24:27])
  submodel_9 = nn.Sequential(n[27:30])
  submodel_10 = nn.Sequential(n[30:])


In [71]:
# extracts the avgpool layer
submodel_avgpool = list(vgg16.children())[1:2]
submodel_avgpool = nn.Sequential(*submodel_avgpool)

In [72]:
submodel_vgg_2 = list(vgg16.children())[2:]
submodel_vgg_2 = nn.Sequential(submodel_flatten, *submodel_vgg_2)
print(submodel_vgg_2)

Sequential(
  (0): Sequential(
    (0): Flatten()
  )
  (1): Sequential(
    (0): Linear(in_features=25088, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )
)


In [ ]:
torch.save(submodel_1, "submodel_1.pt")
torch.save(submodel_2, "submodel_2.pt")
torch.save(submodel_3, "submodel_3.pt")
torch.save(submodel_4, "submodel_4.pt")
torch.save(submodel_5, "submodel_5.pt")
torch.save(submodel_6, "submodel_6.pt")
torch.save(submodel_7, "submodel_7.pt")
torch.save(submodel_8, "submodel_8.pt")
torch.save(submodel_9, "submodel_9.pt")
torch.save(submodel_10, "submodel_10.pt")
torch.save(submodel_avgpool, "submodel_avgpool.pt")
torch.save(submodel_vgg_2, "submodel_vgg_2.pt")

In [ ]:
# Prepare a transform to get the input image into a format (e.g., x,y dimensions) the classifier expects
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])

img = Image.open("input.jpg")

# Apply the transform to the image.
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

with open("classes.txt") as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
# vgg16.eval()
submodel_1.eval()
submodel_2.eval()
submodel_3.eval()
submodel_4.eval()
submodel_5.eval()
submodel_6.eval()
submodel_7.eval()
submodel_8.eval()
submodel_9.eval()
submodel_10.eval()
submodel_avgpool.eval()
submodel_vgg_2.eval()

In [ ]:
# final_output = vgg16(batch_t)
output_submodel_1 = submodel_1(batch_t)
output_submodel_2 = submodel_2(output_submodel_1)
output_submodel_3 = submodel_3(output_submodel_2)
output_submodel_4 = submodel_4(output_submodel_3)
output_submodel_5 = submodel_5(output_submodel_4)
output_submodel_6 = submodel_6(output_submodel_5)
output_submodel_7 = submodel_7(output_submodel_6)
output_submodel_8 = submodel_8(output_submodel_7)
output_submodel_9 = submodel_9(output_submodel_8)
output_submodel_10 = submodel_10(output_submodel_9)
output_submodel_avgpool = submodel_avgpool(output_submodel_10)
final_output = submodel_vgg_2(output_submodel_avgpool)

In [ ]:
# Sort the predictions.
_, indices = torch.sort(final_output, descending=True)

# Convert into percentages.
percentage = torch.nn.functional.softmax(final_output, dim=1)[0] * 100

# Print the 5 most likely predictions.
with open("result_vgg16.txt", "w") as outfile:
    outfile.write(str([(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]))

print(open('result_vgg16.txt', 'r').read()) 